# Chapter 4 : Neural Network Learning

In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 손실 함수 loss function

## 오차 제곱합

$ {1 \over 2}  \displaystyle\sum_{y=y0}^{yn}{(y - t) ^ 2}$

In [2]:
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]

In [3]:
import numpy as np
def sum_squares_error(y, t):
    return 0.5 * np.sum((y-t)**2)

In [4]:
sum_squares_error(np.array(y), np.array(t))

0.09750000000000003

In [7]:
y = [0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0]
sum_squares_error(np.array(y), np.array(t))

0.5975

## 교차 엔트로피 오차

$ E = - \displaystyle\sum_{y=y0}^{yn}{tlogy}$

In [10]:
def cross_entropy_error(y,t):
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

# delta를 더하는 이유? -> y = 0 일 경우엔 log에 들어가면 에러가 난다. 이를 방지하기 위해 매우 작은 수를 더함

In [12]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0]
cross_entropy_error(np.array(y), np.array(t))

0.510825457099338

In [13]:
t = [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]
y = [0.1, 0.05, 0.0, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.6]
cross_entropy_error(np.array(y), np.array(t))

16.11809565095832

## mini-batch learning

mini batch는 왜 사용할까?

-> 훈련 데이터가 많으면 많을 수록 iteration의 시간이 오래 걸림, loss function의 계산에도 부하가 옴

-> 데이터의 일부를 추려 근사치로 이용하도록 함

장점

빠른 시간 내에 적당히 정확한 경로를 찾아낼 수 있음

단점

데이터 전체의 경향을 파악하지 않아 정확하지 않음

**적당히 빠른 길을 찾기 위해 사용됨**

In [3]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from dataset.mnist import load_mnist

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)

x_train.shape
t_train.shape


(60000, 10)

In [4]:
train_size = x_train.shape[0]
batch_size = 10
batch_mask = np.random.choice(train_size, batch_size)
x_batch = x_train[batch_mask]
t_batch = t_train[batch_mask]


print(x_batch.shape[0])
print(x_batch)

10
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size) # flatten answer 
        y = y.reshape(1, y.size) # flatten output 

    batch_size = y.shape[0]
    return -np.sum(t * np.log(y+1e-7)) / batch_size # entropy 

In [6]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size) # flatten answer 
        y = y.reshape(1, y.size) # flatten output

    batch_size = y.shape[0] 
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size
'''
t -> 정답이고 batch_size -> y 갯수로 설정함
np.arange(batch_size)로 0~9 까지 생성시킴
t -> 정답 index 결국 y[0~9 , 정답 num]의 값을 가져와서 log 하는거랑 같음
'''

'\nt -> 정답이고 batch_size -> y 갯수로 설정함\nnp.arange(batch_size)로 0~9 까지 생성시킴\nt -> 정답 index 결국 y[0~9 , 정답 num]의 값을 가져와서 log 하는거랑 같음\n'

loss function을 propagation의 지표로 사용하는 이유

정확도를 기준으로 학습시키면 좋을 것 같지만, 정확도로 학습시키면 시킬수록 미분값이 0이 됨.

즉 학습이 이루어지지 않음

그렇기 때문에 손실함수를 사용함

ex) chapter3 에서의 계단함수와 시그모이드 함수 비교
    계단함수는 대부분의 구간에서 미분값이 0, 시그모이드 함수는 미분값이 거의 항상 변함

In [8]:
def numerical_diff_badcase(f, x):
    h = 1e-50
    return (f(x+h) - f(x)) / h

np.float32(1e-50)

0.0

위 미분 식이 Bad case인 이유

1. np.float32(h)를 했을 때 0이 나옴. 즉, 너무 작은 값을 사용하려하다 보니

    오히려 0으로 취급되어서 오류를 발생시킬 수 도 있음 (반올림 오차 rounding error)

2. 미분은 x위치의 함수의 기울기에 해당함. 
   
   위 함수의 구현은 (x+h)와 x사이의 기울기에 해당하기 때문에 진정한 미분과는 거리가 있음.
   
   즉, h를 0으로 무한하게 좁히는것이 불가능하기 때문에 생기는 한계이다.

In [9]:
def numerical_diff(f, x):
    h = 1e-4
    return (f(x+h) - f(x-h)) / (2*h)

위 중간차분법이 좋은 이유는 증명이 길기 때문에 아래 작성한 벨로그를 참고

h = 1e-4 로 둔 이유는 casting 시 0.0이 되지 않으면서 가장 적은 오차를 내는 h가 1e-4로 알려져 있음